In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- Reference code/ notebook: https://medium.datadriveninvestor.com/k-fold-and-dropout-in-artificial-neural-network-ea054a89fb4e
- https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/
- You can also check my other notebook based on neural network [here](https://www.kaggle.com/nitishkumar3099/neural-network-feature-selection-tps-feb-2022/edit/run/88711391).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode


from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
 
from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')


RANDOM_STATE = 12 
FOLDS = 5

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

In [ ]:
data = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')



In [ ]:

useful_cols = [c for c in data.columns]


useful_cols.remove('row_id')
data = data[useful_cols]
data.duplicated().sum()

In [ ]:
data.drop_duplicates(keep='first', inplace=True)



LABELS = pd.DataFrame()

y = data['target']
LABELS['target'] =y

In [ ]:
useful_col  = [c for c in data.columns]
useful_col.remove('target')
data = data[useful_col]
print(data.shape)

In [ ]:
useful_col = [ c for c in data.columns]
autoscaler = StandardScaler()
data[useful_col] = autoscaler.fit_transform(data[useful_col])

In [ ]:
cols_test = [c for c in test.columns]


cols_test.remove('row_id')
test = test[cols_test]

In [ ]:
cols_test  = [c for c in test.columns]
test[cols_test] = autoscaler.transform(test[cols_test])

In [ ]:
LABELS['target'].value_counts().plot(kind='bar')

print(LABELS['target'].value_counts())

In [ ]:
data = pd.concat([data, LABELS],axis=1)
data.head()

In [ ]:
le = LabelEncoder()
data["target"] = le.fit_transform(data["target"])
target = le.fit_transform(data.target)

display(data.head())
display(data.tail())

In [ ]:
cols = [c for c in data.columns]
cols.remove('target')



In [ ]:
X =data[cols]

In [ ]:
output_category = data['target']

In [ ]:
output_category.head()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers

In [ ]:
# K fold validation
def build_model():
    model=Sequential()
    # first layer 
    # for initializing the weight we use he_normal
    model.add(Dense(units=512,kernel_initializer='he_normal',activation='relu',input_dim=286))
    #2nd layer
    model.add(Dense(units=256,kernel_initializer='he_normal',kernel_regularizer = regularizers.l2(0.001),activation='relu'))
    #3rd layer
    model.add(Dense(units=64,kernel_initializer='he_normal',activation='relu'))
    model.add(Dense(units=512,kernel_initializer='he_normal',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
    # to avoid overfitting we add dropout and kernel_regularizer
    model.add(layers.Dropout(0.5))
    model.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
    model.add(Dense(units = 64, kernel_initializer='he_normal',activation = 'relu'))
    model.add(Dense(units=512,kernel_initializer='random_uniform',kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.002),activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(Dense(units=128,kernel_initializer='he_normal',activation='relu'))
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(Dense(10, kernel_initializer='he_normal',activation = 'softmax'))
    # compiling


    model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
data.shape, X.shape

import numpy as np

k = 5
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = output_category[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [output_category[:i * num_val_samples],
         output_category[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=100, batch_size=512, verbose=0)
    # Evaluate the model on the validation data
    val_scce, val_acc = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_acc)



print(all_scores)
print(np.mean(all_scores))



from keras import backend as K

# Some memory clean-up
K.clear_session()



In [ ]:
import numpy as np

k = 5
num_val_samples = len(X) // k
num_epochs = 200
all_acc_histories = []
all_loss_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = output_category[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [output_category[:i * num_val_samples],
         output_category[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=512, verbose=0)
    accuracy_history = history.history['val_accuracy']
    loss_history = history.history['val_loss']
    all_acc_histories.append(accuracy_history)
    all_loss_histories.append(loss_history)


In [ ]:


average_acc_history = [
    np.mean([x[i] for x in all_acc_histories]) for i in range(num_epochs)]



In [ ]:
average_acc_history

In [ ]:


average_loss_history = [
    np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]




In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,len(average_acc_history)+1), average_acc_history)
plt.xlabel('Epochs')
plt.ylabel('Validation acc')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,len(average_loss_history)+1), average_loss_history)
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.show()

In [ ]:
print(max(average_acc_history))
print(min(average_loss_history))

In [ ]:
#average_acc_history

In [ ]:


average_acc_history_i = [
    [np.mean([x[i] for x in all_acc_histories]) ,i]for i in range(num_epochs)]




In [ ]:


average_loss_history_i = [
    [np.mean([x[i] for x in all_loss_histories]) ,i]for i in range(num_epochs)]





In [ ]:
average_loss_history_i

In [ ]:
average_acc_history_i

#### It seems that the accuracy never increases after epoch = 382.

In [ ]:
min(average_loss_history_i)

In [ ]:
max(average_acc_history_i)

In [ ]:
# Get a fresh, compiled model.
model = build_model()
# Train it on the entirety of the data.
model.fit(X, output_category,
          epochs=58, batch_size=512)


In [ ]:
pred = model.predict(test)

In [ ]:
pred

In [ ]:
sample_submission['target'] = (np.argmax(pred, axis=1)) 
display(sample_submission.head(10))
display(sample_submission.tail(5))

In [ ]:
sample_submission = sample_submission.replace({'target':{0:'Bacteroides_fragilis',1:'Campylobacter_jejuni',2:'Enterococcus_hirae',
                3:'Escherichia_coli',4:'Escherichia_fergusonii',5:'Klebsiella_pneumoniae',6:'Salmonella_enterica',
                    7:'Staphylococcus_aureus',8:'Streptococcus_pneumoniae',9:'Streptococcus_pyogenes'}})

In [ ]:
print(sample_submission['target'].nunique())

In [ ]:
sample_submission.head()

In [ ]:
sub = sample_submission.to_csv('kfold_NeuralNet_TPSFeb2022.csv',index=False)

###### 